In [0]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset

from torchvision.transforms import ToTensor, ToPILImage, Normalize, Compose
import tarfile

import pandas as pd
from zipfile import ZipFile
import io
import cv2 
import os
from google.colab import files
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import *

from PIL import Image
from io import BytesIO

In [0]:
class ImageDataset(Dataset):
  def __init__(self, txt_path='/content/Output_seuillage/small_training_dataset.csv'
, img_dir='/content/Output_seuillage/', transform=None, test=False):
     # initialize variables such is path to csv file and images and transforms
    df = pd.read_csv(txt_path)
   
    categories = {}
    i = 0
    for row in df.species.values:
      if row not in categories:
        categories[row] = i
        i = i + 1
    self.categories = categories
    self.img_names = df.ID.values
    self.format_img = '.jpg'
    self.species = df.species.values
    self.txt_path = txt_path
    self.img_dir = img_dir
    self.transform = transform
    self.to_tensor = ToTensor()
    self.to_pil = ToPILImage()

    self.get_image_selector = True if img_dir.__contains__('tar') else False
    self.tf = tarfile.open(self.img_dir) if self.get_image_selector else None
    self.transform_gt = transform if test else Compose(self.transform.transforms[:-1])  # omit noise of ground truth
    
    print("dataset initialise")
  def __len__(self):
      # here you just need to return a single integer number as the length of your dataset, in your 
      #  case, number of images in your train folder or lines in csv file
    return len(self.img_names)
      #return 1;
  def get_image_from_folder(self, name):
        """
        gets a image by a name gathered from file list text file
        :param name: name of targeted image
        :return: a PIL image
        """
        image_name = os.path.join(self.img_dir, str(name))
        image_name =image_name + str(self.format_img)
        image = Image.open(image_name)
        #image = plt.imread(image_name)
        
        trans1 = transforms.ToTensor()
      
        return trans1(image)
  def getTarget(self, index):
    return self.categories.get(self.species[index])
  def getNameSpecies(self, idDictionary):
    return list(test.keys())[list(test.values()).index(idDictionary)]
  def show(self, index):
    trans = transforms.ToPILImage()
    img = self.get_image_from_folder(self.img_names[index])
    plt.imshow(trans(img))
  def __getitem__(self, index): 
    if index == (self.__len__() - 1) and self.get_image_selector:  # close tarfile opened in __init__
              self.tf.close()
    
    print(self.img_names[index])
    img = self.get_image_from_folder(self.img_names[index])
    #spc = self.species[index]
    spc = self.getTarget(index)
    
    sample = (img, spc)
    

    return sample